In [1]:
# === Standard Library ===
import math
import random
import time
import heapq
import itertools
from collections import defaultdict, deque
from itertools import combinations
from typing import Any, Tuple, Dict, List, Set, Sequence, Union

# === Third-Party Libraries ===

# --- Scientific Computing ---
import numpy as np
import pandas as pd
import scipy.sparse as sp
from scipy.optimize import linprog

# --- Plotting ---
import matplotlib.pyplot as plt

# --- Parallel Processing ---
from joblib import Parallel, delayed
from tqdm import tqdm

# --- Graph Processing ---
import networkx as nx

# --- JIT Compilation ---
from numba import njit, prange

# 1. EPC (parallelized)

In [2]:
def nx_to_csr(G: nx.Graph) -> Tuple[List[int], Dict[int, int], np.ndarray, np.ndarray, np.ndarray]:
     """Convert an undirected NetworkX graph (edge attr `'p'`) to CSR arrays."""
     nodes: List[int] = list(G.nodes())
     idx_of: Dict[int, int] = {u: i for i, u in enumerate(nodes)}

     indptr: List[int] = [0]
     indices: List[int] = []
     probs: List[float] = []

     for u in nodes:
         for v in G.neighbors(u):
             indices.append(idx_of[v])
             probs.append(G.edges[u, v]['p'])
         indptr.append(len(indices))

     return (
         nodes,
         idx_of,
         np.asarray(indptr, dtype=np.int32),
         np.asarray(indices, dtype=np.int32),
         np.asarray(probs, dtype=np.float32),
     )

@njit(inline="always")
def _bfs_component_size(start: int,
                    indptr: np.ndarray,
                    indices: np.ndarray,
                    probs: np.ndarray,
                    deleted: np.ndarray) -> int:
    """Return |C_u|−1 for **one** random realisation (stack BFS)."""
    n = deleted.size
    stack = np.empty(n, dtype=np.int32)
    visited = np.zeros(n, dtype=np.uint8)

    size = 1
    top = 0
    stack[top] = start
    top += 1
    visited[start] = 1

    while top:
        top -= 1
        v = stack[top]
        for eid in range(indptr[v], indptr[v + 1]):
            w = indices[eid]
            if deleted[w]:
                continue
            if np.random.random() >= probs[eid]:
                continue
            if visited[w]:
                continue
            visited[w] = 1
            stack[top] = w
            top += 1
            size += 1
    return size - 1

@njit(parallel=True)
def epc_mc(indptr: np.ndarray,
            indices: np.ndarray,
            probs: np.ndarray,
            deleted: np.ndarray,
            num_samples: int) -> float:
    """Monte‑Carlo estimator of **expected pairwise connectivity** (EPC)."""
    surv = np.where(~deleted)[0]
    m = surv.size
    if m < 2:
        return 0.0

    acc = 0.0
    for _ in prange(num_samples):
        u = surv[np.random.randint(m)]
        acc += _bfs_component_size(u, indptr, indices, probs, deleted)

    return (m * acc) / (2.0 * num_samples)

def epc_mc_deleted(
  G: nx.Graph,
  S: set,
  num_samples: int = 100_000,
) -> float:
  # build csr once
  nodes, idx_of, indptr, indices, probs = nx_to_csr(G)
  n = len(nodes)

  # turn python set S into a mask (node-IDs to delete)
  deleted = np.zeros(n, dtype=np.bool_)
  for u in S:
    deleted[idx_of[u]] = True

  epc = epc_mc(indptr, indices, probs, deleted, num_samples)

  return epc

In [3]:
def component_sampling_epc_mc(G, S, num_samples=1_000,
                              epsilon=None, delta=None, use_tqdm=False):
  """
  Theoretic bounds: compute N = N(epsilon, delta) by the theoretical bound.
  Experimentation:  Otherwise, use the N as input for sample count.
  """

  # Surviving vertex set and its size
  V_remaining = set(G.nodes()) - S
  n_rem = len(V_remaining)

  # base case
  if n_rem < 2:
    return 0.0

  if num_samples is None:
    assert epsilon is not None and delta is not None
    P_E = sum(G.edges[u, v]['p'] for u, v in G.edges())
    coeff = 4 * (math.e - 2) * math.log(2 / delta)
    num_samples = math.ceil(coeff * n_rem * (n_rem - 1) /
                            (epsilon ** 2 * P_E))

  C2 = 0
  it = tqdm(range(num_samples), desc='Component sampling',
            total=num_samples) if use_tqdm else range(num_samples)

  for _ in it:
    u = random.choice(tuple(V_remaining))

    # BFS based on edge probabilities

    visited = {u}
    queue = [u]

    while queue:

      v = queue.pop()
      for w in G.neighbors(v):

        # flip a coin biased by the edge probability
        # w not in deleted nodes
        if w in V_remaining and random.random() < G.edges[v, w]['p']:

          # if w is not visited
          if w not in visited:
              visited.add(w)
              queue.append(w)

    # component counting
    C2 += (len(visited) - 1)

  return (n_rem * C2) / (2 * num_samples)

In [5]:
def epc_celf(G, k, epc_func, **epc_kwargs):
    """
    CELF wrapper to pick k nodes minimizing EPC via lazy greedy.
    """
    # 1) initialize S and baseline EPC
    S = set()
    current_epc = epc_func(G, S, **epc_kwargs)

    # 2) build initial max‐heap of (–gain, node, stamp)
    #    stamp=0 means computed against S at iteration 0
    heap = []
    for v in G.nodes():
        epc_with_v = epc_func(G, S | {v}, **epc_kwargs)
        gain = current_epc - epc_with_v
        heapq.heappush(heap, (-gain, v, 0))

    # 3) lazy‐greedy selection
    iteration = 1
    while len(S) < k:
        neg_gain, v, stamp = heapq.heappop(heap)
        gain = -neg_gain

        if stamp == iteration - 1:
            # this gain is still valid for current S → pick v
            S.add(v)
            current_epc -= gain
            iteration += 1
        else:
            # stale estimate: recompute for the *current* S
            epc_with_v = epc_func(G, S | {v}, **epc_kwargs)
            new_gain = current_epc - epc_with_v
            # push back with updated stamp
            heapq.heappush(heap, (-new_gain, v, iteration - 1))

    return S

In [6]:
delete_set = epc_celf(
    G, 
    k=5, 
    epc_func=component_sampling_epc_mc, 
    num_samples=2_000, 
    use_tqdm=True
)

NameError: name 'G' is not defined

In [ ]:
sizes = [50, 100]
K = 5
results = []

# Generate graphs and compute EPC
for n in sizes:
    G = nx.erdos_renyi_graph(n, p=0.1, seed=42)
    # assign uniform edge probability
    for u, v in G.edges():
        G.edges[u, v]['p'] = 0.7

    S = random.sample(list(G.nodes()), K)

    print(S)
    epc_estimate = epc_mc_deleted(
        G, S, 
        num_samples=1000, use_tqdm=True
        )
  
    delete_set = epc_celf(
        G, 
        k=5, 
        epc_func=epc_mc_deleted, 
        num_samples=2_000, 
        use_tqdm=True
    )
    
    results.append({
        'Graph Size (n)': n,
        'Deletion Set Size (K)': K,
        'EPC Estimate': epc_estimate
    })

# Display results
df = pd.DataFrame(results)

In [7]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)

for u, v in G.edges():
    G.edges[u, v]['p'] = 1.0

K = 10

S = random.sample(list(G.nodes()), K)

print(S)

estimates = []
estimates_cmop = []

for _ in tqdm(range(100)):
    epc_estimate = epc_mc_deleted(
        G.copy(), S,
        num_samples=10000
    )
    estimates.append(epc_estimate)

    epc_comp = component_sampling_epc_mc(
        G.copy(), set(S),
        num_samples=10000
    )
    estimates_cmop.append(epc_comp)

# using NumPy
mean_est = np.mean(estimates)
std_est  = np.std(estimates) 

mean_est_comp = np.mean(estimates_cmop)
std_est_comp  = np.std(estimates_cmop)      # population std (ddof=0)
# std_est = np.std(estimates, ddof=1)  # sample std (ddof=1)

print(f"Mean: {mean_est:.4f}")
print(f"Std:  {std_est:.4f}")

print(f"Mean: {mean_est_comp:.4f}")
print(f"Std:  {std_est_comp:.4f}")

[69, 14, 71, 55, 66, 91, 57, 58, 4, 40]


100%|██████████| 100/100 [01:11<00:00,  1.39it/s]

Mean: 3710.6551
Std:  7.5750
Mean: 3710.4402
Std:  7.5169


In [8]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)

for u, v in G.edges():
    G.edges[u, v]['p'] = 1.0

K = 10

S = random.sample(list(G.nodes()), K)

print(S)

estimates = []
estimates_cmop = []

for _ in tqdm(range(100)):
    epc_estimate = epc_mc_deleted(
        G.copy(), S,
        num_samples=10000
    )
    estimates.append(epc_estimate)

    epc_comp = component_sampling_epc_mc(
        G.copy(), set(S),
        num_samples=10000
    )
    estimates_cmop.append(epc_comp)

# using NumPy
mean_est = np.mean(estimates)
std_est  = np.std(estimates) 

mean_est_comp = np.mean(estimates_cmop)
std_est_comp  = np.std(estimates_cmop)      # population std (ddof=0)
# std_est = np.std(estimates, ddof=1)  # sample std (ddof=1)

print(f"Mean: {mean_est:.4f}")
print(f"Std:  {std_est:.4f}")

print(f"Mean: {mean_est_comp:.4f}")
print(f"Std:  {std_est_comp:.4f}")

[54, 71, 25, 49, 64, 34, 85, 38, 80, 15]


100%|██████████| 100/100 [01:10<00:00,  1.43it/s]

Mean: 3742.0926
Std:  7.2699
Mean: 3740.3395
Std:  7.2448


# 2. GRASP

In [4]:
def grasp_cndp(G: nx.Graph,
               K: int,
               alpha: float = 0.1,
               num_samples: int = 1000,
               restarts: int = 15,
               use_tqdm: bool = False):
    """
    GRASP for Stochastic CNDP:
    """
    best_S, best_score = None, float('inf')

    if use_tqdm:
        it = tqdm(range(restarts), desc="Processing GRASP", total=restarts)
    else:
        it = range(restarts)

    for _ in it:
        S = set()
        # precompute sigma(empty)
        sigma_S = epc_mc_deleted(G, S, num_samples)

        for k in range(K):
            # compute improvement d_j = sigma(S) – sigma(S ∪ {j})
            improvements = {}
            for j in G.nodes():
                if j in S: 
                    continue
                sigma_Sj = epc_mc_deleted(G, S | {j}, num_samples)
                improvements[j] = sigma_S - sigma_Sj

            # find best and worst d
            max_imp = max(improvements.values())
            min_imp = min(improvements.values())

            # build RCL = { j : d_j >= max_imp – alpha*(max_imp – min_imp) }
            threshold = max_imp - alpha * (max_imp - min_imp)
            RCL = [j for j, d in improvements.items() if d >= threshold]

            # pick one at random from RCL
            v = random.choice(RCL)
            S.add(v)

            # update sigma(S)
            sigma_S = epc_mc_deleted(G, S, num_samples)

        if sigma_S < best_score:
            best_score = sigma_S
            best_S = S.copy()

    return best_S, best_score

In [ ]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)
p_edge = 1.0
K = 10
alpha = 0.4

for u, v in G.edges():
  G[u][v]['p'] = p_edge

for alpha in np.arange(0.1, 1.1, 0.1):
  S_star, score_star = grasp_cndp(G, K=K,
                                  alpha=alpha,
                                  num_samples=10_000,
                                  restarts=10)
print("Best removal set:", S_star)
print("Estimated sigma(S*)  :", score_star)

100%|██████████| 20/20 [01:17<00:00,  3.88s/it]


Best removal set: {64, 0, 6, 27, 43, 47, 79, 50, 59, 94}
Estimated σ(S*)  : 3031.1325


100%|██████████| 20/20 [01:17<00:00,  3.87s/it]


Best removal set: {0, 64, 65, 36, 43, 47, 17, 50, 59, 94}
Estimated σ(S*)  : 2975.3595


100%|██████████| 20/20 [01:18<00:00,  3.93s/it]


...

Best removal set: {64, 5, 37, 6, 69, 70, 45, 24, 94, 30}
Estimated σ(S*)  : 3476.484


100%|██████████| 20/20 [01:22<00:00,  4.12s/it]


Best removal set: {96, 0, 47, 49, 19, 83, 86, 55, 90, 62}
Estimated σ(S*)  : 3555.2115


  0%|          | 0/20 [00:02<?, ?it/s]


KeyboardInterrupt: 

## Grasp experiments

In [11]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)
p_edge = 1.0
K = 10
# alpha = 0.4

for u, v in G.edges():
  G[u][v]['p'] = p_edge

for alpha in np.arange(0.1, 0.7, 0.1):

  S_star, score_star = grasp_cndp(G.copy(), K=K,
                                  alpha=alpha,
                                  num_samples=10_000,
                                  restarts=10,
                                  use_tqdm=True)
  
  epc_grasp_final = epc_mc_deleted(G.copy(), S_star, 100_000)

  print("Alpha: ", alpha)
  print("Best removal set:", S_star)
  print("grasp score star", score_star)
  print("Estimated final sigma(S*)  :", epc_grasp_final)

Processing GRASP: 100%|██████████| 10/10 [00:30<00:00,  3.10s/it]


Alpha:  0.1
Best removal set: {0, 64, 69, 6, 43, 47, 50, 57, 59, 94}
grasp score star 3065.283
Estimated final sigma(S*)  : 3080.7621


Processing GRASP: 100%|██████████| 10/10 [00:30<00:00,  3.08s/it]


Alpha:  0.2
Best removal set: {64, 0, 4, 37, 6, 43, 47, 50, 59, 94}
grasp score star 3071.601
Estimated final sigma(S*)  : 3083.7456


Processing GRASP: 100%|██████████| 10/10 [00:31<00:00,  3.11s/it]


Alpha:  0.30000000000000004
Best removal set: {0, 64, 6, 43, 78, 47, 50, 30, 59, 94}
grasp score star 3056.508
Estimated final sigma(S*)  : 3078.6561


Processing GRASP: 100%|██████████| 10/10 [00:31<00:00,  3.18s/it]


Alpha:  0.4
Best removal set: {64, 0, 65, 6, 71, 43, 47, 50, 59, 94}
grasp score star 3151.152
Estimated final sigma(S*)  : 3159.8973


Processing GRASP:   0%|          | 0/10 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [14]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)
p_edge = 1.0
K = 10
# alpha = 0.4

for u, v in G.edges():
  G[u][v]['p'] = p_edge

for alpha in np.arange(0.05, 0.7, 0.05):

  S_star, score_star = grasp_cndp(G.copy(), K=K,
                                  alpha=alpha,
                                  num_samples=10_000,
                                  restarts=10,
                                  use_tqdm=True)
  
  epc_grasp_final = epc_mc_deleted(G.copy(), S_star, 100_000)

  print("Alpha: ", alpha)
  print("Best removal set:", S_star)
  print("grasp score star", score_star)
  print("Estimated final sigma(S*)  :", epc_grasp_final)

Processing GRASP: 100%|██████████| 10/10 [00:54<00:00,  5.47s/it]


Alpha:  0.05
Best removal set: {0, 64, 96, 6, 43, 75, 47, 50, 59, 94}
grasp score star 3072.654
Estimated final sigma(S*)  : 3076.0587


Processing GRASP: 100%|██████████| 10/10 [00:48<00:00,  4.88s/it]


Alpha:  0.1
Best removal set: {64, 0, 6, 27, 43, 12, 47, 50, 59, 94}
grasp score star 3080.025
Estimated final sigma(S*)  : 3082.4469


Processing GRASP: 100%|██████████| 10/10 [00:47<00:00,  4.74s/it]


...

Alpha:  0.55
Best removal set: {0, 3, 6, 39, 71, 47, 81, 19, 84, 27}
grasp score star 3396.8835
Estimated final sigma(S*)  : 3403.94805


Processing GRASP: 100%|██████████| 10/10 [00:57<00:00,  5.70s/it]


Alpha:  0.6000000000000001
Best removal set: {65, 59, 10, 76, 12, 81, 50, 27, 93, 94}
grasp score star 3332.151
Estimated final sigma(S*)  : 3324.0465


Processing GRASP:   0%|          | 0/10 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)
p_edge = 0.9
K = 10
# alpha = 0.4

for u, v in G.edges():
  G[u][v]['p'] = p_edge

for alpha in np.arange(0.05, 0.7, 0.05):

  S_star, score_star = grasp_cndp(G.copy(), K=K,
                                  alpha=alpha,
                                  num_samples=10_000,
                                  restarts=10,
                                  use_tqdm=True)
  
  epc_grasp_final = epc_mc_deleted(G.copy(), S_star, 100_000)

  print("Alpha: ", alpha)
  print("Best removal set:", S_star)
  print("grasp score star", score_star)
  print("Estimated final sigma(S*)  :", epc_grasp_final)

In [13]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)
p_edge = 1.0
K = 10
# alpha = 0.4

for u, v in G.edges():
  G[u][v]['p'] = p_edge

for alpha in np.arange(0.1, 0.7, 0.1):

  S_star, score_star = grasp_cndp(G.copy(), K=K,
                                  alpha=alpha,
                                  num_samples=10_000,
                                  restarts=30,
                                  use_tqdm=True)
  
  epc_grasp_final = epc_mc_deleted(G.copy(), S_star, 100_000)

  print("Alpha: ", alpha)
  print("Best removal set:", S_star)
  print("grasp score star", score_star)
  print("Estimated final sigma(S*)  :", epc_grasp_final)
  

Processing GRASP: 100%|██████████| 30/30 [01:34<00:00,  3.16s/it]


Alpha:  0.1
Best removal set: {64, 0, 6, 27, 43, 12, 47, 50, 59, 94}
grasp score star 3071.952
Estimated final sigma(S*)  : 3080.5164


Processing GRASP: 100%|██████████| 30/30 [01:35<00:00,  3.19s/it]


Alpha:  0.2
Best removal set: {64, 0, 6, 27, 43, 47, 50, 59, 92, 94}
grasp score star 3062.124
Estimated final sigma(S*)  : 3077.0415


Processing GRASP: 100%|██████████| 30/30 [01:33<00:00,  3.12s/it]


Alpha:  0.30000000000000004
Best removal set: {64, 0, 6, 27, 12, 47, 79, 50, 59, 92}
grasp score star 3062.475
Estimated final sigma(S*)  : 3081.0429


Processing GRASP:   0%|          | 0/30 [00:02<?, ?it/s]


KeyboardInterrupt: 

## 2.1 GRASP + local search

In [5]:
def local_search_swap(
    S: Set[int],
    *,
    csr: Tuple[List[int], Dict[int,int], np.ndarray, np.ndarray, np.ndarray],
    num_samples: int = 1000,
    max_iter: int = 1
) -> Set[int]:
    """
    Given initial delete-set S, try 1-for-1 swaps to reduce EPC.
    csr = (nodes, idx_of, indptr, indices, probs).
    """
    nodes, idx_of, indptr, indices, probs = csr
    n = len(nodes)
    deleted = np.zeros(n, dtype=bool)
    for u in S:
        deleted[idx_of[u]] = True

    curr = epc_mc(indptr, indices, probs, deleted, num_samples)

    for _ in range(max_iter):
        best_delta = 0.0
        best_swap = None

        for i in list(S):
            ii = idx_of[i]
            deleted[ii] = False
            for j, jj in idx_of.items():
                if deleted[jj]:
                    continue
                deleted[jj] = True
                sigma_new = epc_mc(indptr, indices, probs, deleted, num_samples)
                delta = curr - sigma_new
                if delta > best_delta:
                    best_delta = delta
                    best_swap = (ii, jj, sigma_new)
                deleted[jj] = False
            deleted[ii] = True

        if best_swap is None:
            break

        ii, jj, curr = best_swap
        deleted[ii] = False
        deleted[jj] = True
        S.remove(nodes[ii])
        S.add(nodes[jj])

    return S

In [6]:
def build_csr(G: nx.Graph):

    nodes = sorted(G.nodes())
    idx_of = {u: i for i, u in enumerate(nodes)}
    n = len(nodes)

    degs = [len(list(G.neighbors(u))) for u in nodes]
    indptr = np.zeros(n + 1, dtype=int)
    indptr[1:] = np.cumsum(degs)

    indices = np.empty(indptr[-1], dtype=int)
    probs = np.empty(indptr[-1], dtype=float)
    ptr = 0

    for u in nodes:
        for v in G.neighbors(u):
            indices[ptr] = idx_of[v]
            probs[ptr] = G.edges[u, v]['p']
            ptr += 1
            
    return nodes, idx_of, indptr, indices, probs

def grasp_with_local_search(
    G: nx.Graph,
    K: int,
    alpha: float = 0.2,
    mc_samples_grasp: int = 1000,
    mc_samples_ls: int = 1000,
    restarts: int = 20,
    max_ls_iter: int = 5
) -> Tuple[Set[int], float]:
    """
    Combined GRASP + local_search_swap procedure.
    """

    csr = build_csr(G)
    best_S, best_score = set(), float('inf')

    for _ in tqdm(range(restarts)):
        S, _ = grasp_cndp(
            G, K, num_samples=mc_samples_grasp, 
            alpha=alpha, restarts=1)
        
        S_imp = local_search_swap(
            S, csr=csr, num_samples=mc_samples_ls, 
            max_iter=max_ls_iter)
        
        score_imp = epc_mc_deleted(
            G, S_imp, 
            num_samples=mc_samples_grasp)
        
        if score_imp < best_score:
            best_score, best_S = score_imp, S_imp.copy()

    return best_S, best_score

## 2.2 GRASP + local search (outside)

- First find best score from GRASP the then utilize the local search

In [7]:
def grasp_with_local_search_outside(
    G: nx.Graph,
    K: int,
    alpha: float = 0.2,
    mc_samples_grasp: int = 10000,
    mc_samples_ls: int = 10000,
    restarts: int = 30,
    max_ls_iter: int = 1,
    use_tqdm: bool = False
) -> Tuple[Set[int], float]:
    """
    Combined GRASP + local_search_swap procedure.
    """

    csr = build_csr(G)
    # best_inner_S, best_inner_score = set(), float('inf')
    best_S, best_score = set(), float('inf')

    S_grasp, epc_grasp = grasp_cndp(
        G, K, num_samples=mc_samples_grasp, 
        alpha=alpha, restarts=restarts, use_tqdm=True)
    
    print(f"\nGrashp EPC: {epc_grasp}\n")
    # score_inner = epc_mc_deleted(
    #     G, S_grasp, 
    #     num_samples=mc_samples_grasp)

    S_last = local_search_swap(
        S_grasp, csr=csr, num_samples=mc_samples_ls, 
        max_iter=max_ls_iter)
    
    score_last = epc_mc_deleted(
            G, S_last, 
            num_samples=mc_samples_grasp)

    return S_last, score_last

## Grasp experiments

In [ ]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)
p_edge = 1.0

K = 10
alpha = 0.2

N_SAMPLE : int = 10000
LS_ITER = 1
grasp_restarts = 20

for u, v in G.edges():
  G[u][v]['p'] = p_edge

S_star_outside, epc_outside = grasp_with_local_search_outside(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_outside_epc_final = epc_mc_deleted(G.copy(), S_star_outside, 100_000)

print("Best removal set:", S_star_outside)
print("Estimated sigma(S*)  :", epc_outside)
print("Estimated final simga(S*)  :", grasp_outside_epc_final)

S_star_ls, epc_grasp_ls = grasp_with_local_search(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_epc_final = epc_mc_deleted(G.copy(), S_star_ls, 100_000)

print("Best removal set:", S_star_ls)
print("Estimated sigma(S*)  :", epc_grasp_ls)
print("Estimated final simga(S*)  :", grasp_epc_final)

Processing GRASP: 100%|██████████| 10/10 [02:39<00:00, 15.98s/it]



Grashp EPC: 3067.038

Best removal set: {0, 64, 6, 27, 43, 12, 47, 50, 59, 94}
Estimated sigma(S*)  : 3089.853
Estimated final simga(S*)  : 3078.621


100%|██████████| 10/10 [11:36<00:00, 69.64s/it]

Best removal set: {64, 0, 34, 67, 43, 79, 50, 24, 59, 94}
Estimated sigma(S*)  : 2965.446
Estimated final simga(S*)  : 3002.90715


In [ ]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)
p_edge = 1.0

K = 10
alpha = 0.2

N_SAMPLE : int = 10000
LS_ITER = 1
grasp_restarts = 20

for u, v in G.edges():
  G[u][v]['p'] = p_edge

S_star_outside, epc_outside = grasp_with_local_search_outside(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_outside_epc_final = epc_mc_deleted(G.copy(), S_star_outside, 100_000)

print("Best removal set:", S_star_outside)
print("Estimated sigma(S*)  :", epc_outside)
print("Estimated final simga(S*)  :", grasp_outside_epc_final)

S_star_ls, epc_grasp_ls = grasp_with_local_search(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_epc_final = epc_mc_deleted(G.copy(), S_star_ls, 100_000)

print("Best removal set:", S_star_ls)
print("Estimated sigma(S*)  :", epc_grasp_ls)
print("Estimated final simga(S*)  :", grasp_epc_final)

Processing GRASP: 100%|██████████| 20/20 [01:02<00:00,  3.12s/it]



Grashp EPC: 3078.27

Best removal set: {64, 0, 6, 40, 43, 47, 50, 59, 94, 31}
Estimated sigma(S*)  : 3082.131
Estimated final simga(S*)  : 3082.2363


100%|██████████| 20/20 [01:51<00:00,  5.59s/it]

Best removal set: {64, 0, 65, 59, 43, 47, 79, 50, 27, 94}
Estimated sigma(S*)  : 3000.0555
Estimated final simga(S*)  : 3001.8951


In [18]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)
p_edge = 0.9

K = 10
alpha = 0.2

N_SAMPLE : int = 10000
LS_ITER = 1
grasp_restarts = 20

for u, v in G.edges():
  G[u][v]['p'] = p_edge

S_star_outside, epc_outside = grasp_with_local_search_outside(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_outside_epc_final = epc_mc_deleted(G.copy(), S_star_outside, 100_000)

print("Best removal set:", S_star_outside)
print("Estimated sigma(S*)  :", epc_outside)
print("Estimated final simga(S*)  :", grasp_outside_epc_final)

S_star_ls, epc_grasp_ls = grasp_with_local_search(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_epc_final = epc_mc_deleted(G.copy(), S_star_ls, 100_000)

print("Best removal set:", S_star_ls)
print("Estimated sigma(S*)  :", epc_grasp_ls)
print("Estimated final simga(S*)  :", grasp_epc_final)

Processing GRASP: 100%|██████████| 20/20 [01:15<00:00,  3.80s/it]



Grashp EPC: 2844.7875

Best removal set: {0, 34, 65, 27, 79, 50, 24, 59, 93, 94}
Estimated sigma(S*)  : 2790.171
Estimated final simga(S*)  : 2794.7664


100%|██████████| 20/20 [02:13<00:00,  6.66s/it]

Best removal set: {0, 65, 64, 27, 47, 79, 50, 59, 93, 94}
Estimated sigma(S*)  : 2887.7265
Estimated final simga(S*)  : 2879.98155


In [20]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)
p_edge = 0.9

K = 10
alpha = 0.2

N_SAMPLE : int = 10000
LS_ITER = 1
grasp_restarts = 20

for u, v in G.edges():
  G[u][v]['p'] = p_edge

S_star_outside, epc_outside = grasp_with_local_search_outside(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_outside_epc_final = epc_mc_deleted(G.copy(), S_star_outside, 100_000)

print("Best removal set:", S_star_outside)
print("Estimated sigma(S*)  :", epc_outside)
print("Estimated final simga(S*)  :", grasp_outside_epc_final)

S_star_ls, epc_grasp_ls = grasp_with_local_search(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_epc_final = epc_mc_deleted(G.copy(), S_star_ls, 100_000)

print("Best removal set:", S_star_ls)
print("Estimated sigma(S*)  :", epc_grasp_ls)
print("Estimated final simga(S*)  :", grasp_epc_final)

Processing GRASP: 100%|██████████| 20/20 [01:15<00:00,  3.77s/it]



Grashp EPC: 2944.7505

Best removal set: {64, 65, 34, 27, 43, 79, 50, 24, 59, 94}
Estimated sigma(S*)  : 2847.8385
Estimated final simga(S*)  : 2853.5589


100%|██████████| 20/20 [02:15<00:00,  6.78s/it]

Best removal set: {64, 0, 34, 6, 43, 79, 50, 24, 27, 94}
Estimated sigma(S*)  : 2938.8555
Estimated final simga(S*)  : 2950.43895


In [22]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)
p_edge = 0.9

K = 10
alpha = 0.2

N_SAMPLE : int = 10000
LS_ITER = 1
grasp_restarts = 30

for u, v in G.edges():
  G[u][v]['p'] = p_edge

S_star_outside, epc_outside = grasp_with_local_search_outside(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_outside_epc_final = epc_mc_deleted(G.copy(), S_star_outside, 100_000)

print("Best removal set:", S_star_outside)
print("Estimated sigma(S*)  :", epc_outside)
print("Estimated final simga(S*)  :", grasp_outside_epc_final)

S_star_ls, epc_grasp_ls = grasp_with_local_search(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_epc_final = epc_mc_deleted(G.copy(), S_star_ls, 100_000)

print("Best removal set:", S_star_ls)
print("Estimated sigma(S*)  :", epc_grasp_ls)
print("Estimated final simga(S*)  :", grasp_epc_final)

Processing GRASP: 100%|██████████| 30/30 [01:53<00:00,  3.79s/it]



Grashp EPC: 2977.8525

Best removal set: {64, 0, 34, 67, 6, 43, 79, 50, 24, 94}
Estimated sigma(S*)  : 2971.4625
Estimated final simga(S*)  : 2965.3452


100%|██████████| 30/30 [03:21<00:00,  6.72s/it]

Best removal set: {64, 0, 65, 43, 47, 79, 50, 27, 92, 94}
Estimated sigma(S*)  : 2917.9665
Estimated final simga(S*)  : 2935.88145


In [23]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)
p_edge = 0.9

K = 10
alpha = 0.2

N_SAMPLE : int = 10000
LS_ITER = 1
grasp_restarts = 30

for u, v in G.edges():
  G[u][v]['p'] = p_edge

S_star_outside, epc_outside = grasp_with_local_search_outside(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_outside_epc_final = epc_mc_deleted(G.copy(), S_star_outside, 100_000)

print("Best removal set:", S_star_outside)
print("Estimated sigma(S*)  :", epc_outside)
print("Estimated final simga(S*)  :", grasp_outside_epc_final)

S_star_ls, epc_grasp_ls = grasp_with_local_search(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_epc_final = epc_mc_deleted(G.copy(), S_star_ls, 100_000)

print("Best removal set:", S_star_ls)
print("Estimated sigma(S*)  :", epc_grasp_ls)
print("Estimated final simga(S*)  :", grasp_epc_final)

Processing GRASP: 100%|██████████| 30/30 [01:54<00:00,  3.80s/it]



Grashp EPC: 3023.4285

Best removal set: {64, 0, 34, 6, 43, 79, 50, 24, 59, 94}
Estimated sigma(S*)  : 2963.3355
Estimated final simga(S*)  : 2966.301


100%|██████████| 30/30 [03:24<00:00,  6.83s/it]

Best removal set: {64, 0, 65, 59, 47, 79, 50, 27, 92, 94}
Estimated sigma(S*)  : 2940.003
Estimated final simga(S*)  : 2942.63955


In [24]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)
p_edge = 0.9

K = 10
alpha = 0.2

N_SAMPLE : int = 10000
LS_ITER = 1
grasp_restarts = 30

for u, v in G.edges():
  G[u][v]['p'] = p_edge

S_star_outside, epc_outside = grasp_with_local_search_outside(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_outside_epc_final = epc_mc_deleted(G.copy(), S_star_outside, 100_000)

print("Best removal set:", S_star_outside)
print("Estimated sigma(S*)  :", epc_outside)
print("Estimated final simga(S*)  :", grasp_outside_epc_final)

S_star_ls, epc_grasp_ls = grasp_with_local_search(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_epc_final = epc_mc_deleted(G.copy(), S_star_ls, 100_000)

print("Best removal set:", S_star_ls)
print("Estimated sigma(S*)  :", epc_grasp_ls)
print("Estimated final simga(S*)  :", grasp_epc_final)

Processing GRASP: 100%|██████████| 30/30 [01:51<00:00,  3.73s/it]



Grashp EPC: 3033.63

Best removal set: {0, 65, 43, 12, 47, 79, 50, 24, 27, 94}
Estimated sigma(S*)  : 2925.9585
Estimated final simga(S*)  : 2934.24885


100%|██████████| 30/30 [03:27<00:00,  6.92s/it]

Best removal set: {0, 64, 65, 59, 43, 47, 79, 50, 27, 94}
Estimated sigma(S*)  : 2941.5915
Estimated final simga(S*)  : 2944.8837


In [19]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)
p_edge = 0.8

K = 10
alpha = 0.2

N_SAMPLE : int = 10000
LS_ITER = 1
grasp_restarts = 20

for u, v in G.edges():
  G[u][v]['p'] = p_edge

S_star_outside, epc_outside = grasp_with_local_search_outside(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_outside_epc_final = epc_mc_deleted(G.copy(), S_star_outside, 100_000)

print("Best removal set:", S_star_outside)
print("Estimated sigma(S*)  :", epc_outside)
print("Estimated final simga(S*)  :", grasp_outside_epc_final)

S_star_ls, epc_grasp_ls = grasp_with_local_search(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_epc_final = epc_mc_deleted(G.copy(), S_star_ls, 100_000)

print("Best removal set:", S_star_ls)
print("Estimated sigma(S*)  :", epc_grasp_ls)
print("Estimated final simga(S*)  :", grasp_epc_final)

Processing GRASP: 100%|██████████| 20/20 [01:21<00:00,  4.07s/it]



Grashp EPC: 2674.3455

Best removal set: {65, 34, 43, 12, 79, 50, 24, 27, 93, 94}
Estimated sigma(S*)  : 2642.5845
Estimated final simga(S*)  : 2639.2617


100%|██████████| 20/20 [02:19<00:00,  6.97s/it]

Best removal set: {0, 65, 34, 59, 91, 79, 49, 24, 27, 93}
Estimated sigma(S*)  : 2648.0925
Estimated final simga(S*)  : 2662.6356


In [62]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)
p_edge = 1.0

K = 10
alpha = 0.1

N_SAMPLE : int = 10000
LS_ITER = 5
grasp_restarts = 20

for u, v in G.edges():
  G[u][v]['p'] = p_edge

S_star_outside, epc_outside = grasp_with_local_search_outside(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_outside_epc_final = epc_mc_deleted(G.copy(), S_star_outside, 100_000)

print("Best removal set:", S_star_outside)
print("Estimated sigma(S*)  :", epc_outside)
print("Estimated final simga(S*)  :", grasp_outside_epc_final)

S_star_ls, epc_grasp_ls = grasp_with_local_search(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_epc_final = epc_mc_deleted(G.copy(), S_star_ls, 100_000)

print("Best removal set:", S_star_ls)
print("Estimated sigma(S*)  :", epc_grasp_ls)
print("Estimated final simga(S*)  :", grasp_epc_final)

Processing GRASP: 100%|██████████| 20/20 [04:51<00:00, 14.57s/it]



Grashp EPC: 3072.303

Best removal set: {0, 64, 96, 6, 43, 75, 47, 50, 59, 94}
Estimated sigma(S*)  : 3085.641
Estimated final simga(S*)  : 3081.3588


100%|██████████| 20/20 [27:06<00:00, 81.34s/it] 


Best removal set: {0, 64, 34, 65, 43, 79, 50, 24, 27, 94}
Estimated sigma(S*)  : 2933.1495
Estimated final simga(S*)  : 2926.8639


In [9]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)
p_edge = 0.9

K = 10
alpha = 0.2

N_SAMPLE : int = 10000
LS_ITER = 1
grasp_restarts = 10

for u, v in G.edges():
  G[u][v]['p'] = p_edge

S_star_outside, epc_outside = grasp_with_local_search_outside(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_outside_epc_final = epc_mc_deleted(G.copy(), S_star_outside, 100_000)

print("Best removal set:", S_star_outside)
print("Estimated sigma(S*)  :", epc_outside)
print("Estimated final simga(S*)  :", grasp_outside_epc_final)

# S_star_ls, epc_grasp_ls = grasp_with_local_search(
#                                 G, K=K,
#                                 alpha=alpha,
#                                 mc_samples_grasp=N_SAMPLE,
#                                 mc_samples_ls=N_SAMPLE,
#                                 restarts=grasp_restarts,
#                                 max_ls_iter=LS_ITER
#                                 )

# grasp_epc_final = epc_mc_deleted(G.copy(), S_star_ls, 100_000)

# print("Best removal set:", S_star_ls)
# print("Estimated sigma(S*)  :", epc_grasp_ls)
# print("Estimated final simga(S*)  :", grasp_epc_final)

Processing GRASP: 100%|██████████| 10/10 [00:34<00:00,  3.47s/it]



Grashp EPC: 3046.4145

Best removal set: {0, 64, 6, 91, 47, 50, 54, 87, 59, 94}
Estimated sigma(S*)  : 3052.962
Estimated final simga(S*)  : 3060.189


In [10]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)
p_edge = 0.9

K = 10
alpha = 0.2

N_SAMPLE : int = 10000
LS_ITER = 1
grasp_restarts = 10

for u, v in G.edges():
  G[u][v]['p'] = p_edge

S_star_outside, epc_outside = grasp_with_local_search_outside(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_outside_epc_final = epc_mc_deleted(G.copy(), S_star_outside, 100_000)

print("Best removal set:", S_star_outside)
print("Estimated sigma(S*)  :", epc_outside)
print("Estimated final simga(S*)  :", grasp_outside_epc_final)

# S_star_ls, epc_grasp_ls = grasp_with_local_search(
#                                 G, K=K,
#                                 alpha=alpha,
#                                 mc_samples_grasp=N_SAMPLE,
#                                 mc_samples_ls=N_SAMPLE,
#                                 restarts=grasp_restarts,
#                                 max_ls_iter=LS_ITER
#                                 )

# grasp_epc_final = epc_mc_deleted(G.copy(), S_star_ls, 100_000)

# print("Best removal set:", S_star_ls)
# print("Estimated sigma(S*)  :", epc_grasp_ls)
# print("Estimated final simga(S*)  :", grasp_epc_final)

Processing GRASP: 100%|██████████| 10/10 [00:36<00:00,  3.67s/it]



Grashp EPC: 3046.041

Best removal set: {0, 64, 34, 6, 43, 79, 50, 24, 59, 94}
Estimated sigma(S*)  : 2959.452
Estimated final simga(S*)  : 2970.5859


In [11]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)
p_edge = 0.9

K = 10
alpha = 0.1

N_SAMPLE : int = 10000
LS_ITER = 1
grasp_restarts = 10

for u, v in G.edges():
  G[u][v]['p'] = p_edge

S_star_outside, epc_outside = grasp_with_local_search_outside(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_outside_epc_final = epc_mc_deleted(G.copy(), S_star_outside, 100_000)

print("Best removal set:", S_star_outside)
print("Estimated sigma(S*)  :", epc_outside)
print("Estimated final simga(S*)  :", grasp_outside_epc_final)

# S_star_ls, epc_grasp_ls = grasp_with_local_search(
#                                 G, K=K,
#                                 alpha=alpha,
#                                 mc_samples_grasp=N_SAMPLE,
#                                 mc_samples_ls=N_SAMPLE,
#                                 restarts=grasp_restarts,
#                                 max_ls_iter=LS_ITER
#                                 )

# grasp_epc_final = epc_mc_deleted(G.copy(), S_star_ls, 100_000)

# print("Best removal set:", S_star_ls)
# print("Estimated sigma(S*)  :", epc_grasp_ls)
# print("Estimated final simga(S*)  :", grasp_epc_final)

Processing GRASP: 100%|██████████| 10/10 [00:35<00:00,  3.58s/it]



Grashp EPC: 3049.5465

Best removal set: {0, 64, 65, 43, 12, 47, 79, 50, 27, 94}
Estimated sigma(S*)  : 2932.5735
Estimated final simga(S*)  : 2943.39285


In [12]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)
p_edge = 0.1

K = 10
alpha = 0.1

N_SAMPLE : int = 10000
LS_ITER = 1
grasp_restarts = 10

for u, v in G.edges():
  G[u][v]['p'] = p_edge

S_star_outside, epc_outside = grasp_with_local_search_outside(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=N_SAMPLE,
                                mc_samples_ls=N_SAMPLE,
                                restarts=grasp_restarts,
                                max_ls_iter=LS_ITER
                                )

grasp_outside_epc_final = epc_mc_deleted(G.copy(), S_star_outside, 100_000)

print("Best removal set:", S_star_outside)
print("Estimated sigma(S*)  :", epc_outside)
print("Estimated final simga(S*)  :", grasp_outside_epc_final)

Processing GRASP: 100%|██████████| 10/10 [00:02<00:00,  4.47it/s]


Grashp EPC: 17.4375

Best removal set: {96, 65, 34, 10, 75, 59, 48, 24, 90, 27}
Estimated sigma(S*)  : 17.3925
Estimated final simga(S*)  : 16.9308


# 3. GRASP + Reactive alpha + Path relinking + LS

In [7]:
class ReactiveAlpha:
    def __init__(self, alpha_vals: List[float]):
        self.alpha_vals = alpha_vals
        self.weights = [1.0] * len(alpha_vals)

    def sample(self) -> Tuple[int, float]:
        total = sum(self.weights)
        r = random.random() * total
        cum = 0.0
        for i, w in enumerate(self.weights):
            cum += w
            if r <= cum:
                return i, self.alpha_vals[i]
        return len(self.weights)-1, self.alpha_vals[-1]

    def reward(self, idx: int, amount: float = 1.0):
        self.weights[idx] += amount

    def penalize(self, idx: int, factor: float = 0.99):
        self.weights[idx] *= factor

def grasp_construct(G: nx.Graph,
                    K: int,
                    alpha: float,
                    mc_samples: int) -> Tuple[Set[int], float]:
    """One GRASP construction (no restarts)."""
    S: Set[int] = set()
    cache: Dict[frozenset, float] = {}

    def sigma(SetS: Set[int]) -> float:
        key = frozenset(SetS)
        if key not in cache:
            cache[key] = epc_mc_deleted(G, SetS, num_samples=mc_samples)
        return cache[key]

    sigma_S = sigma(S)
    for _ in range(K):
        gains = {}
        for v in G.nodes():
            if v in S:
                continue
            gains[v] = sigma_S - sigma(S | {v})
        d_max, d_min = max(gains.values()), min(gains.values())
        thresh = d_max - alpha * (d_max - d_min)
        RCL = [v for v, d in gains.items() if d >= thresh]
        choice = random.choice(RCL)
        S.add(choice)
        sigma_S = sigma(S)

    return S, sigma_S

def local_search_swap(
    S: Set[int],
    *,
    csr: Tuple[List[int], Dict[int,int], np.ndarray, np.ndarray, np.ndarray],
    num_samples: int = 100_000,
    max_iter: int = 5
) -> Set[int]:
    nodes, idx_of, indptr, indices, probs = csr
    n = len(nodes)
    deleted = np.zeros(n, dtype=bool)
    for u in S:
        deleted[idx_of[u]] = True

    curr = epc_mc(indptr, indices, probs, deleted, num_samples)

    for _ in range(max_iter):
        best_delta = 0.0
        best_swap = None
        for i in list(S):
            ii = idx_of[i]
            deleted[ii] = False
            for j in nodes:
                jj = idx_of[j]
                if deleted[jj]:
                    continue
                deleted[jj] = True
                new_sigma = epc_mc(indptr, indices, probs, deleted, num_samples)
                delta = curr - new_sigma
                if delta > best_delta:
                    best_delta = delta
                    best_swap = (ii, jj, new_sigma)
                deleted[jj] = False
            deleted[ii] = True

        if best_swap is None:
            break
        ii, jj, curr = best_swap
        deleted[ii] = False
        deleted[jj] = True
        S.remove(nodes[ii])
        S.add(nodes[jj])

    return S

def path_relink(S: Set[int], E: Set[int],
                G: nx.Graph,
                mc_samples: int) -> Tuple[Set[int], float]:
    """Greedy walk from S toward E, returning best intermediate."""

    T = S.copy()
    best_T, best_score = T.copy(), epc_mc_deleted(G, T, mc_samples)
    D_add = list(E - T)
    D_rm  = list(T - E)

    while D_add and D_rm:
        best_move = None
        best_delta = 0.0
        for i in D_rm:
            for j in D_add:
                T_candidate = T.copy()
                T_candidate.remove(i)
                T_candidate.add(j)
                score = epc_mc_deleted(G, T_candidate, mc_samples)
                delta = best_score - score
                if delta > best_delta:
                    best_delta = delta
                    best_move = (i, j, score)
        if best_move is None:
            break
        i, j, new_score = best_move
        T.remove(i); T.add(j)
        D_rm.remove(i); D_add.remove(j)
        if new_score < best_score:
            best_score = new_score
            best_T = T.copy()

    return best_T, best_score


def insert_into_elite(elite: List[Tuple[Set[int], float]],
                      candidate: Tuple[Set[int], float],
                      max_size: int = 10):
    elite.append(candidate)
    elite.sort(key=lambda x: x[1])
    if len(elite) > max_size:
        elite.pop()

def build_csr(G: nx.Graph):
    nodes = sorted(G.nodes())
    idx_of = {u: i for i, u in enumerate(nodes)}
    degs = [len(list(G.neighbors(u))) for u in nodes]
    indptr = np.zeros(len(nodes)+1, dtype=int)
    indptr[1:] = np.cumsum(degs)
    indices = np.empty(indptr[-1], dtype=int)
    probs   = np.empty(indptr[-1], dtype=float)
    ptr = 0
    for u in nodes:
        for v in G.neighbors(u):
            indices[ptr] = idx_of[v]
            probs[ptr]   = G.edges[u, v]['p']
            ptr += 1
    return nodes, idx_of, indptr, indices, probs


In [8]:
def grasp_meta(
    G: nx.Graph,
    K: int,
    restarts: int = 30,
    mc_samples_grasp: int = 10_000,
    mc_samples_final: int = 10_000,
    mc_samples_ls: int = 10_000,
    max_ls_iter: int = 1,
    elite_size: int = 5,
) -> Tuple[Set[int], float]:
    reactive = ReactiveAlpha([0.05, 0.15, 0.3, 0.5, 0.7, 0.9])
    elite: List[Tuple[Set[int], float]] = []

    # construction only loop
    for _ in tqdm(range(restarts), desc="GRASP construct only", total=restarts):
        idx_alpha, alpha = reactive.sample()

        # build a solution with greedy random adaptive construction
        S_raw, _ = grasp_construct(G, K, alpha, mc_samples_grasp)

        # evaluate the raw solution with a medium sample budget
        score_raw = epc_mc_deleted(G, S_raw, num_samples=mc_samples_final)

        # maintain the elite list
        insert_into_elite(elite, (S_raw, score_raw), max_size=elite_size)

        # reward or penalise that alpha using the raw score
        best_elite_score = elite[0][1]
        
        if score_raw <= best_elite_score:
            reactive.reward(idx_alpha)
        else:
            reactive.penalize(idx_alpha)

    # take the best raw set and polish once
    best_raw_S, _ = elite[0]
    csr = build_csr(G)

    S_final = local_search_swap(
        best_raw_S,
        csr=csr,
        num_samples=mc_samples_ls,
        max_iter=max_ls_iter
    )

    # final high quality evaluation
    final_score = epc_mc_deleted(G, S_final, num_samples=100_000)

    return S_final, final_score

In [ ]:
NODES = 100
p = 0.0443
SEED = 42

H0 = nx.erdos_renyi_graph(NODES, p, seed=SEED)

K = 10
alpha = 0.4
p_edge = 1.0

for u, v in H0.edges():
    H0[u][v]['p'] = p_edge

best_s, best_epc = grasp_meta(H0, K, mc_samples_grasp = 10_000, restarts=30,
                              mc_samples_ls=10_000, mc_samples_pr=10_000,
                              max_ls_iter=5, elite_size=5)

print(f"Best removal set: {best_s}")
print(f"Estimated epc(Best_S): {best_epc}")

Processing grasp meta: 100%|██████████| 50/50 [34:19<00:00, 41.18s/it]

Best removal set: {0, 65, 34, 59, 79, 50, 24, 27, 93, 94}
Estimated epc(Best_S): 2825.055


In [32]:
NODES = 100
p = 0.0443
SEED = 42

H0 = nx.erdos_renyi_graph(NODES, p, seed=SEED)

K = 10
p_edge = 1.0

for u, v in H0.edges():
    H0[u][v]['p'] = p_edge

best_s, best_epc = grasp_meta(H0.copy(), K, restarts=30, mc_samples_grasp = 10_000, 
                              mc_samples_final=10_000, mc_samples_ls=10_000,
                              max_ls_iter=1, elite_size=5)

print(f"Best removal set: {best_s}")
print(f"Estimated epc(Best_S): {best_epc}")

GRASP construct only: 100%|██████████| 30/30 [01:41<00:00,  3.38s/it]


Best removal set: {0, 64, 4, 37, 6, 43, 47, 50, 59, 94}
Estimated epc(Best_S): 3088.8702


In [33]:
NODES = 100
p = 0.0443
SEED = 42

H0 = nx.erdos_renyi_graph(NODES, p, seed=SEED)

K = 10
p_edge = 1.0

for u, v in H0.edges():
    H0[u][v]['p'] = p_edge

best_s, best_epc = grasp_meta(H0.copy(), K, restarts=30, mc_samples_grasp = 10_000, 
                              mc_samples_final=10_000, mc_samples_ls=10_000,
                              max_ls_iter=1, elite_size=5)

print(f"Best removal set: {best_s}")
print(f"Estimated epc(Best_S): {best_epc}")

GRASP construct only: 100%|██████████| 30/30 [01:38<00:00,  3.29s/it]


Best removal set: {64, 0, 6, 27, 43, 47, 79, 50, 59, 94}
Estimated epc(Best_S): 3083.0787


In [34]:
NODES = 100
p = 0.0443
SEED = 42

H0 = nx.erdos_renyi_graph(NODES, p, seed=SEED)

K = 10
p_edge = 0.9

for u, v in H0.edges():
    H0[u][v]['p'] = p_edge

best_s, best_epc = grasp_meta(H0.copy(), K, restarts=30, mc_samples_grasp = 10_000, 
                              mc_samples_final=10_000, mc_samples_ls=10_000,
                              max_ls_iter=1, elite_size=5)

print(f"Best removal set: {best_s}")
print(f"Estimated epc(Best_S): {best_epc}")

GRASP construct only: 100%|██████████| 30/30 [01:51<00:00,  3.70s/it]


Best removal set: {0, 64, 65, 27, 43, 47, 79, 50, 59, 94}
Estimated epc(Best_S): 2943.7038


In [16]:
NODES = 100
p = 0.0443
SEED = 42

H0 = nx.erdos_renyi_graph(NODES, p, seed=SEED)

K = 10
p_edge = 0.8

for u, v in H0.edges():
    H0[u][v]['p'] = p_edge

best_s, best_epc = grasp_meta(H0.copy(), K, restarts=5, mc_samples_grasp = 10_000, 
                              mc_samples_final=10_000, mc_samples_ls=10_000,
                              max_ls_iter=1, elite_size=5)

print(f"Best removal set: {best_s}")
print(f"Estimated epc(Best_S): {best_epc}")

GRASP construct only: 100%|██████████| 5/5 [00:19<00:00,  3.87s/it]


Best removal set: {0, 65, 34, 27, 43, 12, 79, 50, 24, 93}
Estimated epc(Best_S): 2643.0462
